In [3]:
import os
import rasterio

In [5]:
def is_x_constant(raster):
    return raster.xy(0, 0)[0] == raster.xy(raster.height, raster.width)[0]

def is_y_constant(raster):
    return raster.xy(0, 0)[1] == raster.xy(raster.height, raster.width)[1]

files = os.listdir('../data/city_rasters/')
cities = ['aleppo', 'damascus', 'raqqa', 'daraa', 'homs', 'idlib', 'hama', 'deir']
good_files, bad_files = [], []
for city in cities:
    city_files = [file for file in files if city in file]
    for file in city_files:
        raster = rasterio.open(f'../data/city_rasters/{file}')
        x_is_constant, y_is_constant = is_x_constant(raster), is_y_constant(raster)
        
        if x_is_constant or y_is_constant:
            bad_files.append(file)
            
        if not x_is_constant and not y_is_constant:
            good_files.append(file)

On these files, either latitude or the longitude (or both) are equal on the boundaries on the image (we assume it's contant across the entire image). Hence, we cannot use them for linking them with the annotation.

In [6]:
bad_files

['raqqa_2016_07_01_zoom_19.tif',
 'raqqa_2014_03_21_zoom_19.tif',
 'daraa_2014_05_01_zoom_19.tif',
 'daraa_2016_02_25_zoom_19.tif',
 'daraa_2013_11_10_zoom_19.tif',
 'daraa_2016_04_19_zoom_19.tif',
 'homs_2016_05_30_zoom_19.tif',
 'homs_2013_10_31_zoom_19.tif',
 'idlib_2014_05_31_zoom_19.tif',
 'idlib_2014_02_07_zoom_19.tif',
 'hama_2016_07_29_zoom_19.tif',
 'hama_2013_10_31_zoom_19.tif',
 'hama_2014_04_03_zoom_19.tif',
 'deir_2014_09_16_zoom_19.tif',
 'deir_2013_10_24_zoom_19.tif']

On these files, that problem is not present

In [7]:
good_files

['aleppo_2013_05_26_zoom_19.tif',
 'aleppo_2011_06_26_zoom_19.tif',
 'aleppo_2014_07_14_zoom_19.tif',
 'aleppo_2014_05_23_zoom_19.tif',
 'aleppo_2013_09_23_zoom_19.tif',
 'aleppo_2016_09_18_zoom_19.tif',
 'aleppo_2015_10_26_zoom_19.tif',
 'aleppo_2016_10_19_zoom_19.tif',
 'aleppo_2015_11_22_zoom_19.tif',
 'aleppo_2013_10_31_zoom_19.tif',
 'damascus_2011_08_22_zoom_19.tif',
 'damascus_2017_01_22_zoom_19.tif',
 'raqqa_2013_01_17_zoom_19.tif',
 'raqqa_2015_02_02_zoom_19.tif',
 'daraa_2011_10_17_zoom_19.tif',
 'daraa_2017_02_07_zoom_19.tif',
 'homs_2014_04_03_zoom_19.tif',
 'homs_2011_05_21_zoom_19.tif',
 'idlib_2011_07_31_zoom_19.tif',
 'idlib_2016_08_01_zoom_19.tif',
 'hama_2016_06_30_zoom_19.tif',
 'hama_2012_02_22_zoom_19.tif',
 'deir_2016_05_25_zoom_19.tif',
 'deir_2012_12_05_zoom_19.tif']